In [1]:
import torch
import torch.nn as nn
import numpy as np
import random

# tensor
## 新建未初始化張量

In [6]:
a = torch.FloatTensor(3,2)
a

tensor([[2.8586e-36, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [8.9683e-44, 0.0000e+00]])

## tensor 操作
## _結尾 -> inplace
## else -> functional

In [13]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [9]:
torch.FloatTensor([[1,2,3],[3,2,1]])

tensor([[1., 2., 3.],
        [3., 2., 1.]])

In [10]:
n = np.zeros(shape=(3,2))
n

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [11]:
b = torch.tensor(n)
n

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [15]:
n = np.zeros(shape=(3,2), dtype=np.float32)
torch.tensor(n)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [17]:
n = np.zeros(shape=(3,2))
torch.tensor(n, dtype=torch.float32)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

# scalar (零維張量)

In [18]:
a = torch.tensor([1,2,3])
a

tensor([1, 2, 3])

In [20]:
s = a.sum()
s

tensor(6)

In [21]:
s.item()

6

In [22]:
torch.tensor(1)

tensor(1)

# GPU

In [28]:
a = torch.tensor([2,3])
a

tensor([2, 3])

In [25]:
ca = a.cuda()
ca

tensor([2, 3], device='cuda:0')

In [32]:
a = a + 1
a

tensor([5, 6])

In [39]:
ca = a.to('cuda:0')
ca

tensor([5, 6], device='cuda:0')

In [40]:
ca.device

device(type='cuda', index=0)

# gradient
## grad  :  這個屬性包含一個計算出來的"梯度張量"
## is_leaf : 若此張量由使用者建立 -> True
## requires_grad : 若此張量需要計算梯度 -> True

In [41]:
v1 = torch.tensor([1.0,1.0], requires_grad=True)
v2 = torch.tensor([2.0,2.0])

v_sum = v1 + v2
v_sum

tensor([3., 3.], grad_fn=<AddBackward0>)

In [43]:
v_res = (v_sum*2).sum()
v_res

tensor(12., grad_fn=<SumBackward0>)

In [44]:
v1.is_leaf,v2.is_leaf

(True, True)

In [46]:
v_sum.is_leaf,v_res.is_leaf

(False, False)

In [48]:
v1.requires_grad,v2.requires_grad

(True, False)

In [49]:
v_sum.requires_grad,v_res.requires_grad

(True, True)

## 計算梯度
## backward : 計算出相對於計算圖中的其他變數,v_res的導數
## only if "requires_grad=True"

In [51]:
v_res.backward()
v1.grad

tensor([2., 2.])

In [53]:
v2.grad # only see nothing 

# 類神經網路建構模塊
## 建立一個隨機初始化的前饋層

In [54]:
l = nn.Linear(2,5)
v = torch.FloatTensor([1,2])
l(v)

tensor([-1.5072,  0.7072, -0.9237, -0.8783, -0.8142], grad_fn=<AddBackward0>)

## Sequential : 將網路層組合到pipe中

In [57]:
s = nn.Sequential(
    nn.Linear(2,5),
    nn.ReLU(),
    nn.Linear(5,20),
    nn.ReLU(),
    nn.Linear(20,10),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1)
)

s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.3, inplace=False)
  (7): Softmax(dim=1)
)

In [58]:
s(torch.FloatTensor([[1,2]]))

tensor([[0.0999, 0.0877, 0.1016, 0.1404, 0.1044, 0.0877, 0.0877, 0.1151, 0.0877,
         0.0877]], grad_fn=<SoftmaxBackward>)

# 自訂網路層
## 註冊subModule
## 覆寫forward()

In [60]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs,5),
            nn.ReLU(),
            nn.Linear(5,20),
            nn.ReLU(),
            nn.Linear(20,num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        )
    
    def forward(self,x):
        return self.pipe(x)

In [61]:
net = OurModule(num_inputs=2, num_classes=3)
v = torch.FloatTensor([[2,3]])
out = net(v)

In [63]:
net

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=1)
  )
)

In [64]:
out

tensor([[0.4559, 0.2563, 0.2878]], grad_fn=<SoftmaxBackward>)